In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
from gensim import corpora
documents = ["Human machine interface for lab abc computer applications",
                          "A survey of user opinion of computer system response time",
                          "The EPS user interface management system",
                          "System and human system engineering testing of EPS",
                          "Relation of user perceived response time to error measurement",
                          "The generation of random binary unordered trees",
                          "The intersection graph of paths in trees",
                          "Graph minors IV Widths of trees and well quasi ordering",
                          "Graph minors A survey"]
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]

#去掉频次为1的词
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] +=1
texts = [[token for token in text if frequency[token] > 1] for text in texts]
from pprint import pprint
pprint(texts)

D:\Anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-08-08 21:56:10,606 : INFO : 'pattern' package not found; tag filters are not available for English


[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [3]:
dictionary = corpora.Dictionary(texts)
dictionary.save('deerwester.dict')
print(dictionary.token2id)

2018-08-08 21:56:11,381 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-08-08 21:56:11,382 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2018-08-08 21:56:11,383 : INFO : saving Dictionary object under deerwester.dict, separately None
2018-08-08 21:56:11,384 : INFO : saved deerwester.dict


{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [4]:
new_doc = "Human computer interaction"
#doc2bow()统计每个不同单词的出现次数，把词转化成整数词id并且返回稀疏向量结果
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(0, 1), (1, 1)]


In [5]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('deerwester.mm', corpus)
print(corpus)

2018-08-08 21:56:13,990 : INFO : storing corpus in Matrix Market format to deerwester.mm
2018-08-08 21:56:13,992 : INFO : saving sparse matrix to deerwester.mm
2018-08-08 21:56:13,993 : INFO : PROGRESS: saving document #0
2018-08-08 21:56:13,994 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2018-08-08 21:56:13,995 : INFO : saving MmCorpus index to deerwester.mm.index


[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


In [6]:
class MyCorpus(object):
    def __iter__(self):
        for line in open('mycorpus.txt'):
            yield dictionary.doc2bow(line.lower().split())

corpus_memory_friendly = MyCorpus()
for vector in corpus_memory_friendly:
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


In [7]:
from six import iteritems
dictionary = corpora.Dictionary(line.lower().split() for line in open('mycorpus.txt'))
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)
dictionary.compactify() 
print(dictionary)

2018-08-08 21:56:15,813 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-08-08 21:56:15,815 : INFO : built Dictionary(42 unique tokens: ['abc', 'applications', 'computer', 'for', 'human']...) from 9 documents (total 69 corpus positions)


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [8]:
from gensim import corpora, models, similarities
import os
if (os.path.exists("deerwester.dict")):
    dictionary = corpora.Dictionary.load('deerwester.dict')
    corpus = corpora.MmCorpus('deerwester.mm')
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")

2018-08-08 21:56:16,726 : INFO : loading Dictionary object from deerwester.dict
2018-08-08 21:56:16,727 : INFO : loaded deerwester.dict
2018-08-08 21:56:16,729 : INFO : loaded corpus index from deerwester.mm.index
2018-08-08 21:56:16,729 : INFO : initializing cython corpus reader from deerwester.mm
2018-08-08 21:56:16,730 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


Used files generated from first tutorial


In [9]:
#tfidf model
tfidf = models.TfidfModel(corpus)
doc_bow = [(0,1),(1,1)]
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

2018-08-08 21:56:17,531 : INFO : collecting document frequencies
2018-08-08 21:56:17,532 : INFO : PROGRESS: processing document #0
2018-08-08 21:56:17,533 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [10]:
#lsi model
#初始化LSI转换
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
corpus_lsi = lsi[corpus_tfidf] #bow->tfidf->fold-in-lsi
#lsi.print_topics(2)
for doc in corpus_lsi:
    print(doc)
lsi.save('model.lsi')
lsi = models.LsiModel.load('model.lsi')

2018-08-08 21:56:18,313 : INFO : using serial LSI version on this node
2018-08-08 21:56:18,314 : INFO : updating model with new documents
2018-08-08 21:56:18,315 : INFO : preparing a new chunk of documents
2018-08-08 21:56:18,316 : INFO : using 100 extra samples and 2 power iterations
2018-08-08 21:56:18,317 : INFO : 1st phase: constructing (12, 102) action matrix
2018-08-08 21:56:18,318 : INFO : orthonormalizing (12, 102) action matrix
2018-08-08 21:56:18,326 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2018-08-08 21:56:18,327 : INFO : computing the final decomposition
2018-08-08 21:56:18,328 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2018-08-08 21:56:18,329 : INFO : processed documents up to #9
2018-08-08 21:56:18,330 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2018-08-08 21:56:18,331 : INFO : topic #

[(0, 0.06600783396090411), (1, -0.5200703306361846)]
[(0, 0.19667592859142516), (1, -0.760956316770005)]
[(0, 0.08992639972446498), (1, -0.7241860626752505)]
[(0, 0.07585847652178251), (1, -0.6320551586003423)]
[(0, 0.10150299184980113), (1, -0.573730848300296)]
[(0, 0.7032108939378315), (1, 0.16115180214025887)]
[(0, 0.8774787673119835), (1, 0.167589068646595)]
[(0, 0.9098624686818579), (1, 0.14086553628719092)]
[(0, 0.6165825350569281), (1, -0.05392907566389356)]


In [11]:
#similarity interface
dictionary = corpora.Dictionary.load('deerwester.dict')
corpus = corpora.MmCorpus('deerwester.mm')

lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

2018-08-08 21:56:19,258 : INFO : loading Dictionary object from deerwester.dict
2018-08-08 21:56:19,260 : INFO : loaded deerwester.dict
2018-08-08 21:56:19,261 : INFO : loaded corpus index from deerwester.mm.index
2018-08-08 21:56:19,262 : INFO : initializing cython corpus reader from deerwester.mm
2018-08-08 21:56:19,263 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries
2018-08-08 21:56:19,264 : INFO : using serial LSI version on this node
2018-08-08 21:56:19,265 : INFO : updating model with new documents
2018-08-08 21:56:19,266 : INFO : preparing a new chunk of documents
2018-08-08 21:56:19,266 : INFO : using 100 extra samples and 2 power iterations
2018-08-08 21:56:19,267 : INFO : 1st phase: constructing (12, 102) action matrix
2018-08-08 21:56:19,268 : INFO : orthonormalizing (12, 102) action matrix
2018-08-08 21:56:19,269 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2018-08-08 21:56:19,270 : INFO : computing the final decomposition
2018-08-08 2

[(0, 0.4618210045327162), (1, 0.07002766527899945)]


In [17]:
#初始化查询结构
index = similarities.MatrixSimilarity(lsi[corpus])
index.save('deerwester.index')
index = similarities.MatrixSimilarity.load('deerwester.index')
sims = index[vec_lsi]
print(list(enumerate(sims)))

2018-08-08 21:57:42,466 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2018-08-08 21:57:42,468 : INFO : creating matrix with 9 documents and 2 features
D:\Anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
2018-08-08 21:57:42,471 : INFO : saving MatrixSimilarity object under deerwester.index, separately None
2018-08-08 21:57:42,473 : INFO : saved deerwester.index
2018-08-08 21:57:42,474 : INFO : loading MatrixSimilarity object from deerwester.index
2018-08-08 21:57:42,476 : INFO : loaded deerwester.index


[(0, 0.998093), (1, 0.93748635), (2, 0.9984453), (3, 0.9865886), (4, 0.90755945), (5, -0.12416792), (6, -0.10639259), (7, -0.09879464), (8, 0.050041765)]


In [13]:
sims = sorted(enumerate(sims), key = lambda item item: -item[1])
print(sims)

SyntaxError: invalid syntax (<ipython-input-13-d0076a53a276>, line 1)